<a href="https://colab.research.google.com/github/miguelamda/DeepLearningMII/blob/main/Entregables/Ejercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EJERCICIO 2. ENTRENAMIENTO DE MODELOS DE CLASIFICACIÓN PARA DETECTAR MASCARILLAS

En este segundo ejercicio tendrás que probar las técnicas que has visto hasta el módulo 5, incluyendo regularización (L2, dropout, data augmentation, etc.), capas convolucionales, pooling, densas, etc. con **al menos 4 configuraciones** distintas. Además, emplearás tu modelo con tu webcam.

Para ello, vas a tener que trabajar con un conjunto de datos sobre imágenes de personas que llevan o no máscara.

![mask](https://github.com/miguelamda/DeepLearningMII/blob/main/Entregables/img/mask.jpg?raw=1)

## 1. Enunciado

Empleando el dataset de mascarillas (ver apartado 3), debes construir y probar **al menos 4** configuraciones de modelos con PyTorch que sean sustancialmente distintas. Primero, debes hacer cuatro variantes:
1. Dos variantes de **redes convolucionales entrenadas desde cero**. Aquí tendrás que definir (al menos) dos redes con distinta arquitectura, incluyendo: capas convolucionales, pooling, batch normalization, dropout, clasificador (lineal), etc. Simplemente prueba una red menos profunda (menos capas) pero más ancha (más neuronas por capa), y después una red más profunda (más capas) pero menos ancha (menos neuronas por capa). Razona brevemente por qué has considerado cada configuración, y qué esperas conseguir. Considera también el hardware al que tienes acceso, para probar redes más o menos grandes.
2. Dos variantes basadas en **redes convolucionales pre-entrenadas** con transfer learning. Diseña (al menos) dos modelos que usen dos modelos pre-entrenados distintos de TorchVision: uno grande de más de 10M de parámetros (VGG, ResNet, DenseNet, ConvNext, etc.) y uno pequeño de menos de 10M de parámetros (MobileNet, EfficientNet, MNASNet, etc.). Haz extracción de características primero, y luego fine tuning, con cada uno de ellos.

Además, para todas las configuraciones:

* Controla el número de épocas de cada uno usando **early stopping**.
* Usa alguna técnica de **regularización**. Al menos prueba una vez (en alguna configuración) el dropout, L2 y/o el batch normalization.
* Haz **aumentado de datos** en todas las configuraciones. Razona las transformaciones aplicadas, ¿por qué son válidas para este dataset?
* Puedes fijar el optimizador para las cuatro combinaciones, pero juega con el factor de aprendizaje (podría ser distinto para según qué modelo).
* Visualiza con gráficas cómo evoluciona el loss y el accuracy. Se valorará el uso de otras métricas como el precision, el recall o el AUC.

Detalla cada configuración (arquitectura de red, hiperparámetros, etc), haz un resumen de la arquitectura con **torchsummary**. Después, entrénalas con los datos leídos y analiza los resultados, haciendo una comparativa y razonando lo obtenido.

Finalmente, **elige** la mejor configuración a tu criterio, y prepara el **despliegue con PyTorch o con ONNXRuntime**.

## 2. Entrega

La entrega de este ejercicio se realiza a través de la tarea creada para tal efecto en Enseñanza Virtual. Tienes que entregar un **notebook**, y el **HTML** generado a partir de él, cuyas celdas estén ya evaluadas.

La estructura del notebook debe contener los siguientes apartados:

0. Cabecera: nombre y apellidos.
1. Dataset: descripción, carga y visualización.
2. Preparación de los datos para ser usados en PyTorch.
3. Modelos y configuraciones creados en PyTorch (un sub-apartado para cada uno, explicando de forma razonada, con tus palabras y figuras, la arquitectura probada). Entrenamiento y evaluación de cada modelo dentro de su sub-apartado para cada uno.
4. Análisis de resultados.
5. Elección del mejor modelo y despliegue con PyTorch o con ONNXRuntime para predecir las imágenes capturadas con la webcam (ver sección 4). Si no tienes webcam, puedes usar fotos de Internet.
6. Bibliografía utilizada (enlaces web, material de clase, libros, etc.).

### 2.1. Nota importante
-----
**HONESTIDAD ACADÉMICA Y COPIAS: un trabajo práctico es un examen, por lo que
debe realizarse de manera individual. La discusión y el intercambio de
información de carácter general con los compañeros se permite (e incluso se
recomienda), pero NO AL NIVEL DE CÓDIGO. Igualmente el remitir código de
terceros, OBTENIDO A TRAVÉS DE LA RED o cualquier otro medio, se considerará
plagio.**

**Cualquier plagio o compartición de código que se detecte significará
automáticamente la calificación de CERO EN LA ASIGNATURA para TODOS los
alumnos involucrados. Por tanto a estos alumnos NO se les conservará, para
futuras convocatorias, ninguna nota que hubiesen obtenido hasta el momento.
SIN PERJUICIO DE OTRAS MEDIDAS DE CARÁCTER DISCIPLINARIO QUE SE PUDIERAN
TOMAR.**

-----

## 3. El Dataset: Mask Dataset <a class="anchor" id="transferdata"></a>

Este pequeño dataset está disponible en Kaggle. Es una versión reducida del construido en [este tutorial](https://www.pyimagesearch.com/2020/05/04/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/). La pandemia originada por el SARS-Cov2 propició la creación de diversos datasets para detectar si las personas están llevando mascarilla o no. En este dataset para este ejeercicio, tenemos imágenes de personas con y sin mascarilla (algunas se les ha puesto la mascarilla de forma "artificial"). El enlace al dataset es el siguiente [Face Mask Detection](https://www.kaggle.com/omkargurav/face-mask-dataset). Si no tienes cuenta en Kaggle, o si quieres ir más rápido, puedes descargarlo en el siguiente [enlace](https://hdvirtual.us.es/discovirt/index.php/s/dijNRMPYfybYd3N/download).

## 4. Despliegue del modelo pre-entrenado

Una vez hayas entrenado un modelo, puedes lanzar una de las siguientes celdas para hacer la prueba con tu WebCam. No es necesario que adjuntes capturas tuyas, pero sí que se valorará que hagas varias pruebas y analices el ratio de aciertos de tu modelo. Puedes usar también imágenes de internet.

### 4.1 Capturando la WebCam desde Google Colab

Lanza las siguientes dos celdas para capturar una foto con tu web cam, si estás en Google Colab. Haz clic en el botón "Capture" (guardará la imagen en `image.jpg`).

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='image.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

## 4.2 Capturando la WebCam desde local

El siguiente código debería permitirte capturar una foto desde tu webcam si estás en local en un navegador web.

In [ ]:
from IPython.display import HTML, display, update_display

main_text = """
<video id="video" width="320" height="240" autoplay style="display:none"></video>
<canvas id="canvas" width="320" height="240"  ></canvas>

<script>
// Grab elements, create settings, etc.
var video = document.getElementById('video');

// Get access to the camera!
if(navigator.mediaDevices && navigator.mediaDevices.getUserMedia) {
    // Not adding `{ audio: true }` since we only want video now
    navigator.mediaDevices.getUserMedia({ video: true }).then(function(stream) {
        //video.src = window.URL.createObjectURL(stream);
        //video.play();
        video.srcObject=stream;
        video.play();
    });
}

// Elements for taking the snapshot
var canvas = document.getElementById('canvas');
var context = canvas.getContext('2d');
var video = document.getElementById('video');


// Trigger photo take

setInterval(function() {
    context.drawImage(video, 0, 0, 320, 240);
    var myCanvas = document.getElementById('canvas');
    var image = myCanvas.toDataURL("image/png");
    IPython.notebook.kernel.execute("image = '" + image + "'")

}, 2);

</script>

"""
HTML(main_text)

## 4.3 Evaluando el modelo

La siguiente celda debería permitirte cargar la imagen capturada anteriormente por la webcam. Úsala para hacer inferencia con el modelo.

In [ ]:
from PIL import Image
import base64
from numpy import asarray
import io
import numpy as np
import time

IMAGE_SIZE = FIXME # el tamaño de imagen que hayas empleado

#pil_im = Image.open(io.BytesIO(base64.b64decode(image.split(',')[1]))) # versión en local
pil_im = Image.open(filename)  # versión en google colab
pil_im = pil_im.convert('RGB')
pil_im = pil_im.resize(IMAGE_SIZE, Image.ANTIALIAS)
pil_im